In [22]:
import fitz  # PyMuPDF
import re
import logging
import json
import os
import sys
from datetime import datetime
from uuid import uuid4
from typing import List, Dict, Optional
from pathlib import Path
from glob import glob

# Database imports
import psycopg2
from psycopg2.extras import execute_batch
try:
    from pymilvus import Collection, connections, MilvusException
except ImportError:
    MilvusException = Exception  # Fallback

# LangChain imports
try:
    from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
except ImportError as e:
    try:
        from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
    except ImportError:
        logger_init = logging.getLogger(__name__)
        logger_init.warning(f"⚠ LangChain import 실패: {str(e)}")

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [42]:

# ============================================================================
# PDF → Markdown 변환 함수 (계층적 구조 준수)
# ============================================================================

def normalize_legal_markdown(md_text: str) -> str:
    """
    마크다운을 계층적 법률문서 포맷에 맞게 정규화합니다.
    
    지시사항:
    - H1: 법률명 (# 법률명)
    - H2: 장(Chapter) / 부칙 (## 제1장 총칙 또는 ## 부칙)
    - H3: 조(Article) (### 제N조(정의) 또는 ### 제N조의N(가지번호))
    - H4: 항(Paragraph) (#### 제1항 또는 #### 제2항)
    - H5: 부가 정보 (##### 담당부서, ##### 법령정보, ##### 약칭)
    """
    
    lines = md_text.split('\n')
    result = []
    
    # 1. 라인 처리
    for i, line in enumerate(lines):
        original_line = line
        line_stripped = line.strip()
        if not line_stripped:
            result.append('')
            continue
        
        # 이미 마크다운 헤더인 경우 그냥 추가
        if original_line.startswith('# '):
            result.append(original_line)
            continue
        
        # =====================================================
        # 페이지 헤더/푸터 필터링 (무시)
        # =====================================================
        # 패턴: "법제처  1  국가법령정보센터" 또는 "법제처 2 국가법령정보센터"
        # 즉: 한글단어 + 공백 + 숫자 + 공백 + 한글단어 (페이지 번호가 포함)
        if re.match(r'^[가-힣]+\s+\d+\s+[가-힣]+$', line_stripped):
            # 페이지 헤더/푸터이므로 무시
            continue
        
        # =====================================================
        # H5: 부가 정보 분류
        # =====================================================
        
        # 1) 담당부서 정보: "부처명 (부서명) 전화번호"
        # 예: "보건복지부 (간호정책과) 044-202-2691"
        # 예: "국토교통부 (광역교통도로과) 044-201-5121, 5119"
        if re.search(r'[가-힣]+\s*\([^)]+\)\s*\d+-\d+', line_stripped):
            result.append('##### 담당부서')
            result.append(line_stripped)
            continue
        
        # 2) 법령 정보: "[시행 YYYY.M.D.] [법률/대통령령 ...]"
        # 예: "[시행 2025. 6. 21.] [법률 제20445호, 2024. 9. 20., 제정]"
        if re.search(r'^\[시행\s+\d{4}\.\s+\d+\.\s+\d+\.\]\s*\[', line_stripped):
            result.append('##### 법령정보')
            result.append(line_stripped)
            continue
        
        # 3) 약칭 정보: "... ( 약칭: ... )"
        # 예: "간선급행버스체계의 건설 및 운영에 관한 특별법 시행령 ( 약칭: 간선급행버스법 시행령 )"
        if '약칭:' in line_stripped and '(' in line_stripped:
            result.append('##### 약칭')
            result.append(line_stripped)
            continue
        
        # 4) 법제처 헤더: "법제처 ... 국가법령정보센터"
        if '법제처' in line_stripped and '국가법령정보센터' in line_stripped:
            result.append('##### 메타정보')
            result.append(line_stripped)
            continue
        
        # =====================================================
        # H2, H3, H4 헤더 및 본문 처리
        # =====================================================
        
        # 원형 기호(①, ②, ③, ④, ⑤, ⑥) → H4 헤더로 변환
        if '①' in line_stripped:
            line_stripped = re.sub(r'①\s*', '', line_stripped)
            result.append('#### 제1항')
            if line_stripped:
                result.append(line_stripped)
            continue
        elif '②' in line_stripped:
            line_stripped = re.sub(r'②\s*', '', line_stripped)
            result.append('#### 제2항')
            if line_stripped:
                result.append(line_stripped)
            continue
        elif '③' in line_stripped:
            line_stripped = re.sub(r'③\s*', '', line_stripped)
            result.append('#### 제3항')
            if line_stripped:
                result.append(line_stripped)
            continue
        elif '④' in line_stripped:
            line_stripped = re.sub(r'④\s*', '', line_stripped)
            result.append('#### 제4항')
            if line_stripped:
                result.append(line_stripped)
            continue
        elif '⑤' in line_stripped:
            line_stripped = re.sub(r'⑤\s*', '', line_stripped)
            result.append('#### 제5항')
            if line_stripped:
                result.append(line_stripped)
            continue
        elif '⑥' in line_stripped:
            line_stripped = re.sub(r'⑥\s*', '', line_stripped)
            result.append('#### 제6항')
            if line_stripped:
                result.append(line_stripped)
            continue
        
        # 부칙 인식: "[부칙]" 또는 "부칙"
        if line_stripped.startswith('[부칙]') or (line_stripped == '부칙' and not line_stripped.startswith('제')):
            result.append('## 부칙')
            continue
        elif re.match(r'^부칙.*(\d{4}\.\s*\d+\.\s*\d+)', line_stripped):
            result.append('## 부칙')
            version_match = re.search(r'(<.*?>)', line_stripped)
            if version_match:
                result.append(version_match.group(1))
            continue
        
        # 장(Chapter) 인식: "제1장 총칙", "제2장 ..." (H2)
        if re.match(r'^제\d+장\s', line_stripped):
            result.append(f'## {line_stripped}')
            continue
        
        # 조(Article) 인식: "제N조" 또는 "제N조의N" (H3)
        if re.match(r'^제\d+조(의\d+)?\s*\(', line_stripped):
            result.append(f'### {line_stripped}')
            continue
        
        # 삭제된 조문 처리: "제N조 삭제 <날짜>" → 평문 (이탤릭)
        if re.match(r'^제\d+조\s+삭제', line_stripped):
            result.append(f'*{line_stripped}*')
            continue
        
        # 번호 매김 정규화: "1.", "2." 등을 "- 1.", "- 2."로
        if re.match(r'^\d+\.\s', line_stripped):
            line_stripped = re.sub(r'^(\d+\.)(\s)', r'- \1\2', line_stripped)
        
        # 노이즈 제거 - 혼자 떠있는 페이지 번호 등
        if re.match(r'^\d+$', line_stripped):
            continue
        
        result.append(line_stripped)
    
    # 2. 최종 정돈: 여러 빈 줄 → 한두 줄로
    final_text = '\n'.join(result)
    final_text = re.sub(r'\n\n\n+', r'\n\n', final_text)
    
    return final_text


In [43]:

# ============================================================================
# 배치 PDF 처리 함수
# ============================================================================

def batch_pdf_to_markdown(input_dir: str, output_dir: str = "./output/markdown") -> Dict[str, str]:
    """
    디렉토리의 모든 PDF를 마크다운으로 변환합니다.
    """
    
    # 출력 디렉토리 생성
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # PDF 파일 검색
    pdf_files = sorted(glob(os.path.join(input_dir, "*.pdf")))
    
    if not pdf_files:
        logger.warning(f"⚠ PDF 파일을 찾을 수 없습니다: {input_dir}")
        return {}
    
    logger.info(f"📂 발견된 PDF 파일: {len(pdf_files)}개")
    
    results = {}
    successful = 0
    failed = 0
    
    for idx, pdf_path in enumerate(pdf_files, 1):
        pdf_name = Path(pdf_path).name
        logger.info(f"\n[{idx}/{len(pdf_files)}] 처리 중: {pdf_name}")
        
        try:
            # PDF → Markdown 변환
            md_text, metadata = pdf_to_markdown(pdf_path)
            
            if not md_text:
                logger.error(f"  ✗ 변환 실패: {pdf_name}")
                failed += 1
                continue
            
            # Markdown 저장
            base_filename = Path(pdf_path).stem
            md_file_path = save_markdown_to_file(
                md_text,
                output_dir=output_dir,
                filename=f"{base_filename}.md"
            )
            
            if md_file_path:
                results[pdf_name] = md_file_path
                successful += 1
                logger.info(f"  ✓ 저장 완료: {md_file_path}")
            else:
                failed += 1
                
        except Exception as e:
            logger.error(f"  ✗ 오류 발생: {str(e)}")
            failed += 1
    
    logger.info(f"\n{'='*60}")
    logger.info(f"📊 배치 처리 완료:")
    logger.info(f"  - 성공: {successful}/{len(pdf_files)}")
    logger.info(f"  - 실패: {failed}/{len(pdf_files)}")
    logger.info(f"  - 출력: {output_dir}")
    logger.info(f"{'='*60}")
    
    return results


# ============================================================================
# 실행: 배치 처리
# ============================================================================

print("\n" + "="*70)
print("🚀 배치 PDF → Markdown 변환 시작")
print("="*70 + "\n")

# 입력/출력 경로 설정
pdf_input_dir = r"c:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\data"
md_output_dir = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\output"

logger.info(f"📁 입력 디렉토리: {pdf_input_dir}")
logger.info(f"📁 출력 디렉토리: {md_output_dir}\n")

# 배치 처리 실행
try:
    results = batch_pdf_to_markdown(pdf_input_dir, md_output_dir)
    
    if results:
        print("\n📋 변환된 파일 목록:")
        print("-" * 70)
        for pdf_name, md_path in results.items():
            file_size = os.path.getsize(md_path) / 1024
            print(f"  ✓ {pdf_name:30} → {Path(md_path).name} ({file_size:.1f} KB)")
        print("-" * 70)
    else:
        logger.warning("⚠ 변환된 파일이 없습니다.")
    
    print("\n" + "="*70)
    print("✅ 배치 처리 완료!")
    print("="*70)
    
except Exception as e:
    logger.error(f"❌ 배치 처리 중 오류: {str(e)}")
    import traceback
    traceback.print_exc()


2026-01-26 16:39:33,224 - INFO - 📁 입력 디렉토리: c:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\data
2026-01-26 16:39:33,225 - INFO - 📁 출력 디렉토리: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\output

2026-01-26 16:39:33,228 - INFO - 📂 발견된 PDF 파일: 9개
2026-01-26 16:39:33,231 - INFO - 
[1/9] 처리 중: 112신고.pdf
2026-01-26 16:39:33,245 - INFO - ✓ PDF → Markdown 변환 완료: 112신고의 운영 및 처리에 관한 법률 (4 페이지)
2026-01-26 16:39:33,249 - INFO - ✓ 마크다운 파일 저장: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\output\112신고.md (12676 bytes)
2026-01-26 16:39:33,250 - INFO -   ✓ 저장 완료: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\output\112신고.md
2026-01-26 16:39:33,251 - INFO - 
[2/9] 처리 중: 가사근로자.pdf
2026-01-26 16:39:33,260 - INFO - ✓ PDF → Markdown 변환 완료: 가사근로자의 고용개선 등에 관한 법률 시행령 (3 페이지)
2026-01-26 16:39:33,264 - INFO - ✓ 마크다운 파일 저장: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\output\가사근로자.md (10995 bytes)
2026-01-26 16:39:33,265 - INFO -   ✓ 저장 완료: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\output\가사근로자.m


🚀 배치 PDF → Markdown 변환 시작



2026-01-26 16:39:33,406 - INFO - ✓ 마크다운 파일 저장: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\output\개인정보호법.md (175217 bytes)
2026-01-26 16:39:33,410 - INFO -   ✓ 저장 완료: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\output\개인정보호법.md
2026-01-26 16:39:33,410 - INFO - 
[7/9] 처리 중: 법원설치.pdf
2026-01-26 16:39:33,417 - INFO - ✓ PDF → Markdown 변환 완료: 각급 법원의 설치와 관할구역에 관한 법률 (1 페이지)
2026-01-26 16:39:33,421 - INFO - ✓ 마크다운 파일 저장: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\output\법원설치.md (2949 bytes)
2026-01-26 16:39:33,421 - INFO -   ✓ 저장 완료: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\output\법원설치.md
2026-01-26 16:39:33,422 - INFO - 
[8/9] 처리 중: 소비세법.pdf
2026-01-26 16:39:33,452 - INFO - ✓ PDF → Markdown 변환 완료: 개별소비세법 (18 페이지)
2026-01-26 16:39:33,459 - INFO - ✓ 마크다운 파일 저장: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\output\소비세법.md (79189 bytes)
2026-01-26 16:39:33,459 - INFO -   ✓ 저장 완료: C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\output\소비세법.md
2026-01-26 16:39:33,459 - INFO -


📋 변환된 파일 목록:
----------------------------------------------------------------------
  ✓ 112신고.pdf                      → 112신고.md (12.4 KB)
  ✓ 가사근로자.pdf                      → 가사근로자.md (10.7 KB)
  ✓ 간선급버스운행.pdf                    → 간선급버스운행.md (37.2 KB)
  ✓ 간호법.pdf                        → 간호법.md (48.3 KB)
  ✓ 감사원징계.pdf                      → 감사원징계.md (24.1 KB)
  ✓ 개인정보호법.pdf                     → 개인정보호법.md (171.1 KB)
  ✓ 법원설치.pdf                       → 법원설치.md (2.9 KB)
  ✓ 소비세법.pdf                       → 소비세법.md (77.3 KB)
  ✓ 온실가스배출.pdf                     → 온실가스배출.md (15.9 KB)
----------------------------------------------------------------------

✅ 배치 처리 완료!


In [35]:

# 간호법.pdf 첫 페이지 확인
import fitz

test_pdf = r"C:\Users\미소정보기술\OneDrive - 인하대학교\바탕 화면\RAG\test\data\간호법.pdf"
doc = fitz.open(test_pdf)
page = doc[0]
text = page.get_text()
lines = [l.strip() for l in text.split('\n') if l.strip()]

print("=== 간호법.pdf 첫 20줄 ===")
for i, line in enumerate(lines[:20]):
    print(f"{i}: '{line}'")

print("\n현재 추출된 제목:")
title = extract_law_title(text)
print(f"'{title}'")


=== 간호법.pdf 첫 20줄 ===
0: '법제처                                                            1                                                       국가법령정보센터'
1: '간호법'
2: '간호법'
3: '[시행 2025. 6. 21.] [법률 제20445호, 2024. 9. 20., 제정]'
4: '보건복지부 (간호정책과) 044-202-2691'
5: '제1장 총칙'
6: '제1조(목적) 이 법은 모든 국민이 보건의료기관, 학교, 산업현장, 재가 및 각종 사회복지시설 등 간호사등이 종사하는 다'
7: '양한 영역에서 수준 높은 간호 혜택을 받을 수 있도록 간호에 관하여 필요한 사항을 규정함으로써 의료의 질 향상과'
8: '환자안전을 도모하여 국민의 건강 증진에 이바지함을 목적으로 한다.'
9: '제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다.'
10: '1. “간호사”란 제4조에 따른 면허를 받은 사람을 말한다.'
11: '2. “전문간호사”란 제5조에 따른 자격인정을 받은 사람을 말한다.'
12: '3. “간호조무사”란 제6조에 따른 자격인정을 받은 사람을 말한다.'
13: '4. “간호사등”이란 간호사ㆍ전문간호사ㆍ간호조무사를 말한다.'
14: '5. “의료기관”이란 「의료법」 제3조제1항에 따른 의료기관을 말한다.'
15: '6. “보건의료기관”이란 「보건의료기본법」 제3조제4호에 따른 보건의료기관을 말한다.'
16: '제3조(다른 법률과의 관계) 간호사등에 관하여 이 법에서 규정하지 아니한 사항에 대하여는 「보건의료기본법」과 「의료'
17: '법」 등 보건의료 관계 법률을 따른다.'
18: '제2장 면허와 자격'
19: '제4조(간호사 면허) ① 간호사가 되려는 사람은 다음 각 호의 어느 하나에 해당하는 사람으로서 제8조에 따른 간호사'

현재 추출된 제목:
'5. “의료기관”이란 「의료법」 제3조제1항에 따른 의료기관을 말한다.